# 基金-集思录

- 作者：DGC'Idea
- 版本：V0.1   
- 更新日期：2018年3月20日

----------

- 研究目的：获取基金的相关信息；
- 研究方法：从集思录网站爬取基金相关数据。

In [5]:
#导入库
import numpy as np
import pandas as pd
import urllib.request
import json,re,os

#引入自定义库
import sys
sys.path.append('/home/jquser')
from jsl import *

#设置pd显示行列数目
pd.set_option('display.max_rows',500)
pd.set_option('display.max_columns',100)
pd.set_option('display.width',2000)
pd.set_option('max_colwidth',200)
# pd.set_option('colheader_justify', 'left')
#指数ETF
def get_index_etf(data_type):
    df=get_jsl_data(data_type)
    df.index=df['fund_id']
    df.index.name=None
    df=df[['fund_nm',
           'price','increase_rt','volume',
           'estimate_value','fund_nav','nav_dt',
           'discount_rt','creation_unit',
           'amount','unit_incr','unit_total',
           'index_nm'#,'index_id'
          ]]
    df=df.rename(columns={'fund_nm':'名称','price':'现价','increase_rt':'涨幅','volume':'成交额(万)',
                          'estimate_value':'估值','fund_nav':'净值','nav_dt':'净值日期',
                          'discount_rt':'溢价率','creation_unit':'最小申赎(万)',
                          'amount':'份额(万)','unit_incr':'规模变化(亿)','unit_total':'规模(亿)',
                          'index_nm':'指数'#,'index_id':'代码'
                         }) 
    return df

#分级基金
def get_m_sfund(data_type):
    df=get_jsl_data(data_type)
    df.index=df['base_fund_id']
    df.index.name=None
#     df=df[['fund_nm',
#            'price','increase_rt','volume',
#            'estimate_value','fund_nav','nav_dt',
#            'discount_rt',
#            'amount','unit_incr','unit_total',
#            'chinese_nm',
#           ]]
#     df=df.rename(columns={'fund_nm':'名称','price':'现价','increase_rt':'涨幅','volume':'成交额(万)',
#                           'estimate_value':'估值','fund_nav':'净值','nav_dt':'净值日期',
#                           'discount_rt':'溢价率',
#                           'amount':'份额(万)','unit_incr':'规模变化(亿)','unit_total':'规模(亿)',
#                           'chinese_nm':'标的',
#                          }) 
    return df

#分级基金
def get_b_sfund(data_type):
    df=get_jsl_data(data_type)
    df.index=df['funda_id']
    df.index.name=None
#     df=df[['fund_nm',
#            'price','increase_rt','volume',
#            'estimate_value','fund_nav','nav_dt',
#            'discount_rt',
#            'amount','unit_incr','unit_total',
#            'chinese_nm',
#           ]]
#     df=df.rename(columns={'fund_nm':'名称','price':'现价','increase_rt':'涨幅','volume':'成交额(万)',
#                           'estimate_value':'估值','fund_nav':'净值','nav_dt':'净值日期',
#                           'discount_rt':'溢价率',
#                           'amount':'份额(万)','unit_incr':'规模变化(亿)','unit_total':'规模(亿)',
#                           'chinese_nm':'标的',
#                          }) 
    return df


#分级基金
def get_a_sfund(data_type):
    df=get_jsl_data(data_type)
    df.index=df['funda_id']
    df.index.name=None
#     df=df[['fund_nm',
#            'price','increase_rt','volume',
#            'estimate_value','fund_nav','nav_dt',
#            'discount_rt',
#            'amount','unit_incr','unit_total',
#            'chinese_nm',
#           ]]
#     df=df.rename(columns={'fund_nm':'名称','price':'现价','increase_rt':'涨幅','volume':'成交额(万)',
#                           'estimate_value':'估值','fund_nav':'净值','nav_dt':'净值日期',
#                           'discount_rt':'溢价率',
#                           'amount':'份额(万)','unit_incr':'规模变化(亿)','unit_total':'规模(亿)',
#                           'chinese_nm':'标的',
#                          }) 
    return df


#stock_lof
def get_stock_lof(data_type):
    df=get_jsl_data(data_type)
    df.index=df['fund_id']
    df.index.name=None
    df['apply_redeem_status']=df['apply_redeem_status'].str.replace('限','')
    df=df[[
        'fund_nm','price','increase_rt','volume',
        'amount','amount_incr','turnover_rt',
        'fund_nav','estimate_value','discount_rt',
        'stock_ratio','stock_increase_rt',
        'apply_fee','redeem_fee','apply_redeem_status',
          ]]
    df=df.rename(columns={
        'fund_nm':'名称','price':'现价','increase_rt':'涨幅','volume':'成交(万)',
        'amount':'场内份额(万)','amount_incr':'场内新增(万)','turnover_rt':'换手率',
        'fund_nav':'基金净值','estimate_value':'实时估值','discount_rt':'溢价率',
        'stock_ratio':'股票占比','stock_increase_rt':'重仓涨幅',
        'apply_fee':'申购费','redeem_fee':'赎回费','apply_redeem_status':'申赎状态',
                         }) 
    return df

#index_lof
def get_index_lof(data_type):
    df=get_jsl_data(data_type)
    df.index=df['fund_id']
    df.index.name=None
    df=df[[
        'fund_nm','price','increase_rt','volume',
        'amount','amount_incr','turnover_rt',
        'fund_nav','estimate_value','discount_rt',
        'index_nm','index_increase_rt',
        'apply_fee','redeem_fee','apply_redeem_status',
          ]]
    df=df.rename(columns={
        'fund_nm':'名称','price':'现价','increase_rt':'涨幅','volume':'成交(万)',
        'amount':'场内份额(万)','amount_incr':'场内新增(万)','turnover_rt':'换手率',
        'fund_nav':'基金净值','estimate_value':'实时估值','discount_rt':'溢价率',
        'index_nm':'跟踪指数','index_increase_rt':'指数涨幅',
        'apply_fee':'申购费','redeem_fee':'赎回费','apply_redeem_status':'申赎状态',
                         }) 
    return df

#黄金etf
def get_gold_etf(data_type):
    df=get_jsl_data(data_type)
    df.index=df['fund_id']
    df.index.name=None
    df=df[['fund_nm',
           'price','increase_rt','volume',
           'chinese_nm','index_increase_rt',
           'estimate_value','fund_nav','nav_dt',
           'discount_rt',
           'amount','unit_incr','unit_total',
          ]]
    df=df.rename(columns={'fund_nm':'名称','price':'现价','increase_rt':'涨幅','volume':'成交额(万)',
                          'chinese_nm':'标的','index_increase_rt':'标的涨幅',
                          'estimate_value':'估值','fund_nav':'净值','nav_dt':'净值日期',
                          'discount_rt':'溢价率',
                          'amount':'份额(万)','unit_incr':'规模变化(亿)','unit_total':'规模(亿)',
                         }) 
    return df

#T+0 QDII
def get_t0_qdii(data_type):
    df=get_jsl_data(data_type)
    df.index=df['fund_id']
    df.index.name=None
    df=df[['fund_nm',
        'price','increase_rt','volume',
        'amount','amount_incr','turnover_rt',                     
        'fund_nav','nav_dt',
        'estimate_value','nav_dt','est_val_increase_rt',
        'discount_rt',
        'index_nm','ref_increase_rt',
        'apply_fee','redeem_fee',
          ]]
    df=df.rename(columns={'fund_nm':'名称','price':'现价','increase_rt':'涨幅','volume':'成交额(万)',
                          'amount':'场内份额(万)','amount_incr':'场内新增(万)','turnover_rt':'换手率',                     
                          'fund_nav':'净值','nav_dt':'净值日期',
                          'estimate_value':'T-1估值','nav_dt':'净值日期','est_val_increase_rt':'估值涨幅',
                          'discount_rt':'溢价率',
                          'index_nm':'标的','ref_increase_rt':'标的涨幅',
                          'apply_fee':'申购费','redeem_fee':'赎回费',
                         }) 
    return df



## 1.指数ETF

In [3]:
df=get_index_etf('index_etf')
df.sort_values(['溢价率'],ascending=False)

,名称,现价,涨幅,成交额(万),估值,净值,净值日期,溢价率,最小申赎(万),份额(万),规模变化(亿),规模(亿),指数
512210,景顺食品,2.140,0.00%,0.00,2.0452,2.0946,2018-03-06,4.43%,150,300,0.00,0.06,中证800食品
159909,深TMT,5.590,-0.09%,0.61,5.3877,5.4460,2018-03-21,3.62%,25,1352,0.00,0.76,TMT50
510260,新兴ETF,1.212,1.85%,1.73,1.1838,1.1890,2018-03-21,2.33%,50,5600,0.00,0.68,上证新兴
159929,医药ETF,1.706,-1.50%,234.39,1.6691,1.6890,2018-03-21,2.16%,50,6170,0.00,1.05,中证医药
159930,能源ETF,0.836,0.24%,8.20,0.8185,0.8189,2018-03-21,2.09%,50,1440,0.00,0.12,中证能源
512230,景顺医药,1.435,0.00%,0.00,1.4063,1.4231,2018-03-06,2.00%,50,300,0.00,0.04,中证医药
159908,深F200,1.541,-0.90%,0.40,1.5177,1.5375,2018-03-21,1.51%,150,3573,0.00,0.55,深证F200
510380,国寿300,1.015,0.00%,0.00,1.0010,1.0111,2018-03-21,1.38%,90,12100,0.00,1.23,沪深300
159944,全指材料,0.938,1.19%,2.47,0.9263,0.9286,2018-03-21,1.25%,200,6361,0.00,0.60,全指材料
512010,医药ETF,1.835,-0.22%,328.12,1.8123,1.8313,2018-03-21,1.24%,100,7500,0.02,1.38,300医药


## 2.黄金ETF

In [11]:
df=get_gold_etf('gold_etf')
df.sort_values(['溢价率'],ascending=False)

,名称,现价,涨幅,成交额(万),标的,标的涨幅,估值,净值,净值日期,溢价率,份额(万),规模变化(亿),规模(亿)
159937,博时黄金,2.712,0.86%,4379.00,黄金9999,0.86%,2.7159,2.6927,2018-03-21,-0.14%,163302,1.90,44.29
518800,黄金基金,2.692,0.82%,7584.38,黄金9999,0.86%,2.6944,2.6714,2018-03-21,-0.09%,8300,0.03,2.23
159934,黄金ETF,2.698,0.78%,3026.87,黄金9999,0.86%,2.6994,2.6764,2018-03-21,-0.05%,45166,0.08,12.19
518880,黄金ETF,2.713,0.86%,87913.73,黄金9999,0.86%,2.7144,2.6913,2018-03-21,-0.05%,170700,0.16,46.31


## 3.指数LOF

In [12]:
df=get_index_lof('index_lof')
df.sort_values(['溢价率'],ascending=False)

,名称,现价,涨幅,成交(万),场内份额(万),场内新增(万),换手率,基金净值,实时估值,溢价率,跟踪指数,指数涨幅,申购费,赎回费,申赎状态
501010,生物科C,1.236,3.52%,7.16,139,-9,4.31%,1.1852,1.1716,5.50%,中证生科,-1.21%,0,0.10%,开放/开放
167901,华宸300,1.270,0.00%,0.00,61,0,0.00%,1.2550,1.2431,2.16%,沪深300,-1.00%,1.20%,1.50%,开放申购/开放赎回
166007,中欧300,1.452,0.00%,0.00,63,0,0.00%,1.4384,1.4247,1.92%,沪深300,-1.00%,1.20%,0.50%,开放/开放
501012,中药C,1.044,1.36%,1.37,167,0,0.78%,1.0328,1.0249,1.86%,中证中药,-0.81%,0,0.10%,开放/开放
501007,互联医疗,1.006,0.00%,7.76,588,-3,1.31%,0.9986,0.9892,1.70%,CS互医疗,-0.99%,0.80%,0.10%,开放/开放
501020,国企改,1.040,-0.67%,2.41,490,0,0.47%,1.0328,1.0237,1.59%,国企改革,-0.93%,1.2%,0.5%,暂停/开放
162510,国安双力,1.030,0.29%,0.12,729,0,0.02%,1.0230,1.0188,1.10%,中小板综,-0.43%,1.20%,0.50%,开放/开放
160635,医药基金,1.150,-0.95%,4.25,101,0,3.67%,1.1540,1.1411,0.78%,中证医药,-1.18%,1.20%,0.50%,开放/开放
501002,能源互联,0.952,-0.42%,0.21,220,0,0.10%,0.9440,0.9460,0.63%,CS能源互,0.22%,1.20%,0.50%,开放/开放
161033,智能汽车,1.005,-1.28%,13.21,805,-9,1.62%,1.0110,0.9997,0.53%,CS智汽车,-1.18%,1.20%,0.50%,开放/开放


## 4.股票LOF

In [16]:
df=get_stock_lof('stock_lof')
df.sort_values(['溢价率'],ascending=False)

,名称,现价,涨幅,成交(万),场内份额(万),场内新增(万),换手率,基金净值,实时估值,溢价率,股票占比,重仓涨幅,申购费,赎回费,申赎状态
161219,国投产业,1.375,0.00%,0.00,76,0,0.00%,1.3360,1.3293,3.44%,62.03%,-0.50%,1.50%,0.50%,开放/开放
501001,财通精选,1.225,0.66%,40.08,5924,-51,0.56%,1.2070,1.1892,3.01%,84.67%,-1.48%,1.20%,0.50%,开放/开放
160613,鹏华创新,1.242,0.08%,0.19,132,0,0.11%,1.2340,1.2161,2.13%,94.66%,-1.45%,1.50%,0.50%,开放/开放
161222,国投瑞利,1.235,0.00%,0.00,3753,0,0.00%,1.2270,1.2151,1.64%,71.59%,-0.97%,1.50%,1.50%,开放/开放
501017,国泰融丰,0.914,0.00%,0.00,26072,-29,0.00%,0.9017,0.8998,1.58%,22.83%,-0.21%,1.5%,1.5%,开放/开放
160106,南方高增,1.283,-0.54%,8.29,6073,0,0.11%,1.2770,1.2632,1.57%,83.15%,-1.08%,1.50%,0.50%,开放/开放
163503,天治核心,0.494,0.00%,2.59,10770,1,0.05%,0.4901,0.4865,1.54%,94.00%,-0.73%,1.50%,0.50%,开放/开放
160610,鹏华动力,0.990,0.10%,11.61,4927,0,0.24%,0.9830,0.9756,1.48%,78.69%,-0.76%,1.50%,0.50%,开放/开放
165516,信诚周期,2.107,-1.27%,0.21,173,0,0.06%,2.0930,2.0771,1.44%,78.48%,-0.76%,1.50%,0.50%,开放/开放
169101,东证睿丰,1.564,-0.95%,3339.35,46082,322,4.61%,1.5610,1.5461,1.16%,85.55%,-0.96%,1.5%,1.5%,大额/开放


## 5.T+0 QDII

In [10]:
df=get_t0_qdii('t0_qdii')
df.sort_values(['溢价率'],ascending=False)

,名称,现价,涨幅,成交额(万),场内份额(万),场内新增(万),换手率,净值,净值日期,T-1估值,净值日期,估值涨幅,溢价率,标的,标的涨幅,申购费,赎回费
161116,易基黄金,0.693,0.14%,18.30,4104,0,0.64%,0.6840,2018-03-20,0.6829,2018-03-20,-0.17%,1.48%,伦敦金价格,-0.17%,0,0.2%
160719,嘉实黄金,0.692,0.29%,68.15,5218,4,1.89%,0.6950,2018-03-21,0.6837,2018-03-21,-0.19%,1.21%,伦敦金价格,-0.19%,0.8%,0.5%
164701,添富贵金,0.597,0.17%,8.17,3338,0,0.41%,0.5910,2018-03-20,0.5903,2018-03-20,-0.12%,1.14%,伦敦金价格收益率,-0.12%,0.8%,0.5%
160216,国泰商品,0.473,1.94%,301.83,17537,-14,3.64%,0.4800,2018-03-21,0.4682,2018-03-21,0.68%,1.03%,国泰大宗商品配置指数,0.68%,1.5%,0.5%
501023,港中小企,1.163,-1.02%,0.15,1902,-25,0.01%,1.1741,2018-03-21,1.1598,2018-03-21,-1.22%,0.28%,港中小企,-0.91%,1.2%,0.5%
160416,石油基金,0.925,1.65%,73.44,6879,-3,1.15%,0.9170,2018-03-20,0.9228,2018-03-20,0.64%,0.24%,标普全球石油指数,0.64%,0,0.5%
159941,纳指100,1.444,0.14%,7.77,1305,0,0.41%,1.4448,2018-03-20,1.4415,2018-03-20,-0.23%,0.17%,纳斯达克100,-0.23%,-,-
513500,标普500,1.600,0.13%,47.84,27200,0,0.11%,1.5964,2018-03-20,1.5974,2018-03-20,0.06%,0.16%,标普500,0.06%,-,-
501021,香港中小,1.468,-0.88%,372.10,24218,4,1.04%,1.4842,2018-03-21,1.4664,2018-03-21,-1.20%,0.11%,香港中小,-0.89%,1.0%,0.5%
513100,纳指ETF,2.288,-0.26%,200.25,12100,0,0.72%,2.2860,2018-03-21,2.2867,2018-03-21,-0.23%,0.06%,纳斯达克100,-0.23%,-,-


## 6.分级A

In [15]:
df=get_a_sfund('a_sfund')
df

,abrate,base_est_dis_rt_tip,calc_info,coupon_descr,coupon_descr_s,fund_descr,funda_amount,funda_amount_increase,funda_amount_increase_rt,funda_base_est_dis_rt,funda_base_est_dis_rt_t1,funda_base_est_dis_rt_t2,funda_base_fund_id,funda_coupon,funda_coupon_next,funda_current_price,funda_discount_rt,funda_id,funda_increase_rt,funda_index_id,funda_index_increase_rt,funda_index_name,funda_left_year,funda_lower_recalc_rt,funda_name,funda_nav_dt,funda_profit_rt,funda_profit_rt_next,funda_value,funda_volume,fundb_upper_recalc_rt,fundb_upper_recalc_rt_info,last_time,left_recalc_year,lower_recalc_profit_rt,next_recalc_dt,owned,price_tips,status_cd
150223,5:5,,,固定6.0%,6.0%,每年12月15日定折，遇节假日提前,184013,545,0.30%,0.21%,0.24%,0.06%,161027,6.00,6.00,0.987,2.76%,150223,0.20%,399975,-0.10%,证券公司,永续,25.08%,证券A级,2018-03-21,6.17%,6.173%,1.0150,5688.27,77.68%,NaN,15:02:03,0.73425,1.69%,2018-12-15,0,现价:0.987; 最后更新时间:15:02:03,N
150188,7:3,,,3年定存+利差(0.5%-2%)，本期利差1.25%，每三年周期折算后调整,其它,每年12月15日定折，遇节假日顺延，每3年周期折算，分折为母基金再拆分，周期折算时B停牌,4460,0,0.00%,0.26%,0.74%,0.77%,161719,4.50,4.50,0.986,2.57%,150188,0.00%,000832,-0.33%,标普转债,1.73,6.41%,转债优先,2018-03-21,6.11%,6.107%,1.0120,0.41,55.35%,NaN,15:02:03,0.73425,0.30%,2018-12-15,0,现价:0.986; 最后更新时间:15:02:03,N
150106,5:5,,,固定7.0%,7.0%,09/20，周年内最后一个工作日定折,6248,8,0.13%,1.67%,0.71%,-0.67%,161118,7.00,7.00,1.047,-1.21%,150106,-0.10%,399005,-0.85%,中小板指,1.50,41.67%,中小A,2018-03-21,6.07%,6.068%,1.0345,65.68,81.63%,NaN,15:02:03,0.49863,-,2018-09-20,0,现价:1.047; 最后更新时间:15:02:03,N
150221,5:5,,,+5.0%,+5.0%,每年第一个工作日定折,41768,-260,-0.62%,-0.55%,-0.40%,-0.59%,164402,6.50,6.50,1.081,-7.24%,150221,-0.46%,399959,-0.60%,军工指数,永续,43.26%,中航军A,2018-03-21,6.06%,6.058%,1.0080,368.90,35.30%,NaN,15:02:03,0.78356,-5.32%,2019-01-02,0,现价:1.081; 最后更新时间:15:02:03,N
150321,5:5,,,+5.0%,+5.0%,每年12月第一个工作日,2551,0,0.00%,0.64%,0.04%,0.09%,161032,6.50,6.50,1.098,-7.75%,150321,1.86%,399998,-0.15%,中证煤炭,永续,42.81%,煤炭A基,2018-03-21,6.02%,6.024%,1.0190,23.91,35.21%,NaN,15:02:03,0.70137,-5.79%,2018-12-03,0,现价:1.098; 最后更新时间:15:02:03,N
150331,5:5,,,+4.5%,+4.5%,每年12月第一个工作日定折,2886,-36,-1.23%,0.01%,0.56%,-0.41%,165315,6.00,6.00,1.009,-0.21%,150331,0.00%,399805,-0.10%,互联金融,永续,42.57%,网金融A,2018-03-22,5.99%,5.987%,1.0071,11.10,37.07%,NaN,15:02:03,0.70137,-0.52%,2018-12-03,0,现价:1.009; 最后更新时间:15:02:03,N
150123,5:5,,,+4.5%,+4.5%,每年第一个工作日定折,7056,-75,-1.05%,-0.32%,-0.43%,-0.74%,165312,6.00,6.00,1.016,-0.28%,150123,0.10%,399550,-1.62%,央视50,永续,34.69%,建信50A,2018-03-22,5.98%,5.983%,1.0133,94.86,106.81%,NaN,15:02:03,0.78356,-0.62%,2019-01-02,0,现价:1.016; 最后更新时间:15:02:03,N
150297,5:5,,,+4.0%,+4.0%,每年12月1日定折，节假日顺延，基金合同生效不足6个月的除外，2015年不定折，每次定折调整约定收益率,1323,-10,-0.75%,-4.99%,0.65%,-1.03%,160137,5.50,5.50,0.942,7.34%,150297,-9.07%,H30535,-0.57%,互联网,永续,21.55%,互联A级,2018-03-21,5.94%,5.943%,1.0166,0.25,85.80%,NaN,15:02:03,0.70137,5.83%,2018-12-03,0,现价:0.942; 最后更新时间:15:02:03,N
150022,5:5,,,+3.0%,+3.0%,每年第一个工作日定折，无下折，A不参与上折，净值<1元无定折，母基净值连续10个交易日超过2.0上折,175626,230,0.13%,0.08%,0.20%,-0.47%,163109,4.50,4.50,0.772,23.56%,150022,-0.64%,399001,-0.94%,深证成指,永续,5.16%,深成指A,2018-03-22,5.91%,5.905%,1.0100,456.28,241.86%,深成指A不参与上折,15:02:03,0.78356,-,"<span style=""font-style:italic"">2019-01-02</span>",0,现价:0.772; 最后更新时间:15:02:03,N
150301,5:5,,,+4.0%,+4.0%,每年12月15日定折，节假日顺延，A类不参与上折,3290,-1,-0.03%,0.87%,-0.02%,0.07%,160419,5.50,5.50,0.948,6.55%,150301,0.11%,399975,-0.10%,证券公司,永续,27.86%,证券股A,2018-03-21,5.89%,5.892%,1.0145,0.95,71.16%,证券股A不参与上折,15:02:03,0.73425,4.95%,2018-12-15,0,现价:0.948; 最后更新时间:15:02:03,N


## 7.分级B

In [2]:
df=get_b_sfund('b_sfund')
df

,abrate,asset_ratio_last,b_est_val,base_est_dis_rt_tip,calc_info,coupon,coupon_descr,coupon_descr_s,fundB_amount,fundB_amount_increase,fundB_amount_increase_rt,funda_current_price,funda_id,funda_increase_rt,funda_lower_price,funda_name,funda_ratio,funda_upper_price,fundb_base_est_dis_rt,fundb_base_est_dis_rt_tip,fundb_base_fund_id,fundb_base_price,fundb_capital_rasising_rt,fundb_current_price,fundb_discount_rt,fundb_discount_rt2,fundb_id,fundb_increase_rt,fundb_index_id,fundb_index_increase_rt,fundb_index_name,fundb_left_year,fundb_lower_price,fundb_lower_recalc_rt,fundb_name,fundb_nav_dt,fundb_net_leverage_rt,fundb_price_leverage_rt,fundb_ratio,fundb_unit_capital_rasising_rt,fundb_upper_price,fundb_upper_recalc_rt,fundb_value,fundb_volume,fundm_nav_dt,fundm_value,last_time,manage_fee,maturity_dt,owned,recalc_to,status_cd,upper_recalc_price
150032,8:2,-,0.952,NaN,,5.000,固定5.0%,5.0%,606,0,-0.08%,1.028,150032,0.00,0.925,多利优先,8,1.131,0.47%,,160718,1.0298,6.03%,1.061,11.43%,2.72,150033,0.00%,399923,0.00%,债券总指,永续,0.955,10.72%,多利进取,2018-03-21,5.407,4.853,2,1.56%,1.167,-,0.9522,0.00,2018-03-21,1.0298,15:02:03,0.900,None,0,None,N,None
150188,7:3,0%,0.643,NaN,,4.500,3年定存+利差(0.5%-2%)，本期利差1.25%，每三年周期折算后调整,其它,1911,0,0.02%,0.986,150188,0.00,0.887,转债优先,7,1.085,0.26%,,161719,0.9040,5.89%,0.711,10.65%,2.91,150189,-3.27%,000832,-0.33%,标普转债,1.73,0.662,6.41%,转债进取,2018-03-21,4.675,4.225,3,1.83%,0.809,55.35%,0.6520,0.26,2018-03-21,0.9040,15:02:03,1.000,2019-12-15,0,None,N,1.40
150289,5:5,99%,0.408,NaN,,5.500,+4.0%,+4.0%,54056,76,0.14%,0.964,150289,0.31,0.865,煤炭A级,5,1.057,0.21%,,168204,0.7120,7.53%,0.461,13.00%,5.30,150290,0.66%,399998,-0.15%,中证煤炭,永续,0.412,11.11%,煤炭B级,2018-03-21,3.485,3.085,5,3.61%,0.504,110.97%,0.4090,1944.86,2018-03-21,0.7120,15:02:03,1.220,None,0,None,N,1.50
150164,7:3,53%,1.026,NaN,,4.500,+3.0%,+3.0%,493,-12,-2.47%,0.946,150164,-3.86,0.886,可转债A,7,1.082,-1.08%,,165809,1.0170,5.43%,1.146,10.62%,5.14,150165,1.33%,000832,-0.33%,中证转债,永续,1.018,17.00%,可转债B,2018-03-21,3.272,2.958,3,2.78%,1.244,37.66%,1.0360,10.29,2018-03-22,1.0170,15:02:03,0.500,None,0,None,N,1.40
150022,5:5,95%,0.160,NaN,,4.500,+3.0%,+3.0%,175626,229,0.13%,0.772,150022,-0.64,0.699,深成指A,5,0.855,0.05%,,163109,0.5852,7.68%,0.399,133.74%,23.90,150023,-1.72%,399001,-0.94%,深证成指,永续,0.365,5.19%,深成指B,2018-03-21,6.856,2.933,5,3.98%,0.447,241.76%,0.1707,1022.87,2018-03-22,0.5852,15:02:03,1.220,None,0,None,N,2.00
150094,5:5,85%,0.414,NaN,,5.000,+3.5%,+3.5%,362,2,0.51%,0.959,150094,0.00,0.863,泰信400A,5,1.055,2.25%,,162907,0.7120,7.27%,0.497,20.34%,8.40,150095,-9.64%,000966,-0.54%,基本400,永续,0.495,11.45%,泰信400B,2018-03-22,3.448,2.865,5,3.90%,0.605,180.90%,0.4130,0.05,2018-03-22,0.7120,15:02:03,1.220,None,0,None,N,2.00
150143,7:3,53%,1.085,NaN,,4.500,+3.0%,+3.0%,1026,0,0.01%,0.932,150143,0.00,0.839,转债A级,7,1.025,-1.73%,,161826,1.0380,6.58%,1.215,11.93%,5.57,150144,1.67%,000832,-0.33%,中证转债,永续,1.076,18.42%,转债B级,2018-03-21,3.178,2.839,3,3.58%,1.315,44.96%,1.0960,1.87,2018-03-21,1.0380,15:02:03,1.220,None,0,None,N,1.50
150275,5:5,0%,0.454,NaN,,4.500,+3.0%,+3.0%,27300,0,0.00%,0.931,150275,0.11,0.837,一带一A,5,1.023,-0.07%,,167503,0.7330,6.72%,0.534,17.62%,8.00,150276,-0.93%,399991,-0.29%,一带一路,永续,0.485,13.92%,一带一B,2018-03-22,3.229,2.745,5,3.85%,0.593,104.64%,0.4540,84.91,2018-03-22,0.7330,15:02:03,1.200,None,0,None,N,1.50
150297,5:5,95%,0.598,NaN,,5.500,+4.0%,+4.0%,1323,-10,-0.74%,0.942,150297,-9.07,0.932,互联A级,5,1.140,-4.99%,,160137,0.8117,7.30%,0.592,-1.00%,-0.60,150298,-1.00%,H30535,-0.57%,互联网,永续,0.538,21.55%,互联B级,2018-03-21,2.700,2.727,5,4.23%,0.658,85.80%,0.6068,6.60,2018-03-21,0.8117,15:02:03,1.220,None,0,None,N,1.50
150117,5:5,100%,0.543,NaN,,5.500,+4.0%,+4.0%,43168,0,0.00%,0.951,150117,0.21,0.854,房地产A,5,1.044,-0.05%,,160218,0.7864,7.74%,0.603,11.07%,6.00,150118,-3.37%,399393,-1.21%,国证地产,永续,0.562,18.84%,房地产B,2018-03-21,2.864,2.578,5,4.90%,0.686,157.28%,0.5610,1180.19,2018-03-21,0.7864,15:02:03,1.200,None,0,None,N,2.00


## 8.分级母基金

In [6]:
df=get_m_sfund('m_sfund')
df

,base_fund_id,base_fund_nm,base_fund_py,active_fl,mtrade,market,mflag,fund_company_nm,fund_manager,issue_dt,maturity_dt,list_dt,manage_fee,index_id,index_nm,upper_recalc_price,lower_recalc_price,asset_ratio,fully,a_ratio,b_ratio,fund_descr,money_cd,pre_recacl_dt,next_recalc_dt,next_recalc_dt2,fundA_id,fundA_nm,fundA_py,coupon_descr,coupon_descr_s,coupon,coupon_next,coupon_next2,fundB_id,fundB_nm,fundB_py,lower_recalc_comment,a_amount,a_amount_increase_rt,status_cd,recalc_to,calc_info,apply_fee,apply_fee_tip,redeem_fee,redeem_fee_tip,min_apply_amount,notes,price,last_chg_dt,b_gangan,base_lower_recalc_rt,base_est_dis_rt,left_year,abrate,base_est_dis_rt_tip
160127,160127,南方消费,nfxf,Y,N,sz,Y,南方基金,蒋秋洁,2012-03-13,-,2012-04-25,1.750,399942,主动基金,定折归一,0.20,80.000,Y,5,5,03/13，定折时A/B/M全归1，可顺延定折日,CNY,None,2019-03-13,None,150049,消费收益,xfsy,+3.2%，动态调整,+3.2%,4.70,4.70,None,150050,消费进取,xfjq,B类净值低于,None,None,N,None,,1.5%,100万以下，1.5%\r\n100万（含）至500万，0.9%\r\n500万（含）至1000万，0.3%\r\n1000万（含）以上，每笔1000元\r\n,0.5%,持有1年以下，0.5%\r\n持有1年至2年，0.3%\r\n持有2年以上，0,None,http://www.nffund.com/new/fundproduct/?action=searchjjfl&fundcode=160127&index=2,0.9730,2018-03-21,2.059,38.28%,0.72%,永续,5:5,
160135,160135,高铁基金,gtjj,Y,N,sz,Y,南方基金,雷俊,2015-06-10,-,2015-06-23,1.200,399807,高铁产业,1.50,0.25,95.000,Y,5,5,每年12月1日定折，遇节假日顺延，基金合同生效不足6个月的除外，2015年不定折，每次定折调整约定收益率,CNY,None,2018-12-03,None,150293,高铁A级,gtaj,+4.0%,+4.0%,5.50,5.50,None,150294,高铁B级,gtbj,B类净值低于,None,None,N,None,,0,0,0.5%,持有1年以下，0.50%\n持有1年至2年，0.25%\n持有2年以上，0,None,http://www.nffund.com/new/fundproduct/?action=searchjjfl&fundcode=160135&index=2,0.8952,2018-03-21,2.314,29.26%,3.50%,永续,5:5,
160136,160136,改革基金,ggjj,Y,N,sz,Y,南方基金,雷俊,2015-06-03,-,2015-06-15,1.200,399974,国企改革,1.50,0.25,95.000,Y,5,5,每年12月1日定折，遇节假日顺延，基金合同生效或不定期折算后不足 6 个月的除外,CNY,None,2018-12-03,None,150295,改革A,gga,+3.5%,+3.5%,5.00,5.00,None,150296,改革B,ggb,B类净值低于,None,None,N,None,,0,0,0.5%,持有1年以下，0.50%\n持有1年至2年，0.25%\n持有2年以上，0,None,http://www.nffund.com/new/fundproduct/?action=searchjjfl&fundcode=160136&index=2,0.9896,2018-03-21,2.072,35.51%,0.01%,永续,5:5,
160137,160137,互联基金,hljj,Y,N,sz,Y,南方基金,柯晓,2015-07-01,-,2015-07-13,1.220,H30535,互联网,1.50,0.25,95.000,Y,5,5,每年12月1日定折，节假日顺延，基金合同生效不足6个月的除外，2015年不定折，每次定折调整约定收益率,CNY,None,2018-12-03,None,150297,互联A级,hlaj,+4.0%,+4.0%,5.50,5.50,None,150298,互联B级,hlbj,B类净值低于,None,None,N,None,,0,0,0.5%,持有1年以下，0.50%\n持有1年至2年，0.25%\n持有2年以上，0,None,http://www.nffund.com/new/fundproduct/?action=searchjjfl&fundcode=160137&index=2,0.8117,2018-03-21,2.700,21.55%,-4.99%,永续,5:5,
160217,160217,国泰互利,gthl,Y,N,sz,Y,国泰基金,吴晨,2011-12-29,-,2012-01-17,0.900,399481,中证全债,1.60,0.40,90.000,Y,7,3,每年第一个工作日定折,CNY,None,2019-01-02,None,150066,互利A,hla,+1.5%，动态调整,+1.5%,3.00,3.00,None,150067,互利B,hlb,B类净值低于,None,None,N,None,,0.8%,100万以下，0.8%\r\n100万（含）至300万，0.5%\r\n300万（含）至500万元，0.3%\r\n500万元（含）以上，每笔1000元,0.1%,持有1年以下，0.1%\r\n持有1年（含）至2年，0.05%\r\n持有2年（含）以上，0,None,http://www.gtfund.com/product/productlist/zhaiquan/160217/index.html\r\n,1.0600,2018-03-21,2.978,22.26%,0.55%,永续,7:3,
160218,160218,国泰地产,gtdc,Y,N,sz,Y,国泰基金,徐皓,2013-02-06,-,2013-03-07,1.200,399393,国证地产,2.00,0.25,95.000,Y,5,5,每年第一个工作日定折，每次定折或不定折基准日变更利率,CNY,None,2019-01-02,None,150117,房地产A,fdca,+4.0%,+4.0%,5.50,5.50,None,150118,房地产B,fdcb,B类净值低于,None,None,N,None,,0,0,0.5%,0.5%,None,http://www.gtfund.com/product/productlist/zhishu/160218/index.html\r\n,0.7864,2018-03-21,2.864,18.84%,-0.05%,永续,5:5,
160219,160219,国泰医药,gtyy,Y,N,sz,Y,国泰基金,邱晓华,2013-10-15,-,2013-09-11,1.220,399394,国证医药,1.50,0.25,95.000,Y,5,5,每年第一个工作日定折，每次定折或不定折基准日变更利率,CNY,None,2019-01-02,None,150130,医药A,yya,+4.0%,+4.0%,5.50,5.50,None,150131,医药B,yyb,B类净值低于,None,None,N,None,,0,0,0.5%,0.5%,None,http://www.gtfund.com/product/productlist/zhishu/160219/index.html\r\n,0.8900,2018-03-21,2.353,28.29%,-0.26%,永续,5:5,
160221,160221,国泰有色,gtys,Y,N,sz,Y,国泰基金,徐皓,2015-03-30,-,2015-04-15,1.220,399395,国证有色,1.50,0.25,95.000,Y,5,5,每年第一个工作日定折，每次定折或不定折基准日变更利率,CNY,None,2019-01-02,None,150196,有色A,ysa,+4.0%,+4.0%,5.50,5.50,None,150197,有色B,ysb,B类净值低于,None,None,N,None,,0,0,0.7%,持有1年以下，0